# Streaming with MLflow ResponsesAgent

This notebook demonstrates streaming capabilities in MLflow ResponsesAgent for real-time response generation.

## Table of Contents
1. Why Use Streaming?
2. Streaming Interface Overview
3. Basic Text Streaming
4. Streaming with Tool Calls
5. Advanced Streaming Patterns
6. Production Considerations

## Setup

In [ ]:
import os
from dotenv import load_dotenv
from typing import Generator

import mlflow
from mlflow.entities.span import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
)
from openai import OpenAI

# Load environment
load_dotenv()
assert "OPENAI_API_KEY" in os.environ, "Please set OPENAI_API_KEY in .env file"

# Set experiment
mlflow.set_experiment("Streaming_ResponseAgent")

print(f"MLflow version: {mlflow.__version__}")
print("✅ Setup complete!")

## 1. Why Use Streaming?

### Benefits of Streaming:

1. **Better User Experience**: Users see responses as they're generated
2. **Reduced Perceived Latency**: First token appears quickly
3. **Real-time Processing**: Handle long-running operations gracefully
4. **Progress Visibility**: Show intermediate steps in agent workflows
5. **Resource Efficiency**: Process data incrementally

### Streaming vs Non-Streaming:

| Aspect | Non-Streaming | Streaming |
|--------|---------------|----------|
| Response | Wait for complete response | Receive chunks incrementally |
| Latency | High perceived latency | Low perceived latency |
| Memory | Hold entire response | Process incrementally |
| UI Updates | Single update | Continuous updates |
| Tool Calls | See after completion | See as they happen |

## 2. Streaming Interface Overview

### Key Streaming Types:

**ResponsesAgentStreamEvent**: The wrapper for all streaming events

```python
ResponsesAgentStreamEvent(
    type="response.output_text.delta",  # Event type
    delta={...},                          # For delta events
    item={...},                           # For done events
)
```

### Event Types:

| Event Type | Purpose |
|------------|--------|
| `response.output_text.delta` | Text chunk |
| `response.output_item.done` | Complete output item |
| `response.annotation.added` | Annotation event |

### Helper Methods for Streaming:

1. `create_text_delta(delta, item_id)` - Create text chunk event
2. `create_annotation_added(...)` - Create annotation event
3. `create_text_output_item(...)` - Create complete text output

## 3. Basic Text Streaming

Let's implement basic text streaming in a ResponsesAgent.

In [ ]:
class BasicStreamingAgent(ResponsesAgent):
    """
    Demonstrates basic text streaming with ResponsesAgent.
    
    Key concepts:
    - yield text deltas with same item_id
    - yield final output_item.done to complete
    """
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """
        Non-streaming prediction.
        Collects stream events and returns final response.
        """
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs)
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        Streaming prediction - yields text chunks.
        
        Pattern:
        1. Yield text deltas with consistent item_id
        2. Yield output_item.done with complete text
        """
        # Simulate generating text in chunks
        text_chunks = [
            "Hello! ",
            "I am a streaming ",
            "ResponsesAgent. ",
            "This demonstrates ",
            "real-time text generation!"
        ]
        
        full_text = ""
        item_id = "msg_stream_1"
        
        # Yield each chunk as a delta event
        for chunk in text_chunks:
            full_text += chunk
            yield ResponsesAgentStreamEvent(
                **self.create_text_delta(delta=chunk, item_id=item_id)
            )
        
        # Yield the complete output item
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item(
                text=full_text,
                id=item_id,
            ),
        )


# Test basic streaming
print("Testing basic streaming agent...\n")
agent = BasicStreamingAgent()

# Stream the response
print("Streaming output:")
print("-" * 40)
for event in agent.predict_stream({"input": [{"role": "user", "content": "Hi!"}]}):
    if event.type == "response.output_text.delta":
        print(f"[DELTA] {event.delta}", end="", flush=True)
    elif event.type == "response.output_item.done":
        print(f"\n\n[DONE] Complete item received")
        
print("\n" + "-" * 40)
print("✅ Basic streaming complete!")

## 4. Streaming with OpenAI Integration

Let's integrate real OpenAI streaming with ResponsesAgent.

In [ ]:
from mlflow.types.responses import (
    output_to_responses_items_stream,
    to_chat_completions_input,
)


class OpenAIStreamingAgent(ResponsesAgent):
    """
    ResponsesAgent that streams from OpenAI ChatCompletions API.
    
    Uses MLflow's conversion utilities to handle format translation.
    """
    
    def __init__(self, model: str = "gpt-4o-mini"):
        self.model = model
        self.client = OpenAI()
    
    def call_llm_stream(self, messages: list):
        """Stream responses from OpenAI."""
        for chunk in self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            stream=True,
        ):
            yield chunk.to_dict()
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """Non-streaming prediction."""
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(
            output=outputs,
            custom_outputs=request.custom_inputs
        )
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        Stream from OpenAI and convert to ResponsesAgent format.
        
        Uses output_to_responses_items_stream for automatic conversion.
        """
        # Convert input to ChatCompletions format
        messages = to_chat_completions_input(
            [i.model_dump() for i in request.input]
        )
        
        # Stream and convert
        yield from output_to_responses_items_stream(
            self.call_llm_stream(messages)
        )


# Enable OpenAI autologging
mlflow.openai.autolog()

# Test OpenAI streaming
print("Testing OpenAI streaming agent...\n")
openai_agent = OpenAIStreamingAgent(model="gpt-4o-mini")

test_request = {
    "input": [{"role": "user", "content": "Write a haiku about Python programming."}]
}

print("Streaming from OpenAI:")
print("-" * 50)

for event in openai_agent.predict_stream(test_request):
    if hasattr(event, 'delta') and event.delta:
        print(event.delta, end="", flush=True)
    elif event.type == "response.output_item.done":
        print("\n\n[Stream Complete]")

print("-" * 50)
print("✅ OpenAI streaming complete!")

## 5. Streaming with Tool Calls

Streaming tool calls allows users to see agent actions in real-time.

In [ ]:
import json
import time


class StreamingToolAgent(ResponsesAgent):
    """
    Demonstrates streaming with tool calls.
    
    Pattern:
    1. Yield function_call as output_item.done
    2. Yield function_call_output as output_item.done
    3. Yield final text response
    """
    
    def execute_tool(self, name: str, args: dict) -> str:
        """Execute tool and return result."""
        if name == "get_time":
            import datetime
            return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        elif name == "calculate":
            op = args.get("operation")
            x, y = args.get("x", 0), args.get("y", 0)
            if op == "add":
                return str(x + y)
            elif op == "multiply":
                return str(x * y)
        return "Unknown tool"
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs)
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        Stream tool calls and results.
        """
        # Simulate: Agent decides to call a tool
        tool_call = {
            "name": "calculate",
            "arguments": json.dumps({"operation": "multiply", "x": 7, "y": 6})
        }
        
        # 1. Yield the function call
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_function_call_item(
                id="fc_1",
                call_id="call_123",
                name=tool_call["name"],
                arguments=tool_call["arguments"],
            ),
        )
        
        # 2. Execute tool and yield result
        args = json.loads(tool_call["arguments"])
        result = self.execute_tool(tool_call["name"], args)
        
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_function_call_output_item(
                call_id="call_123",
                output=result,
            ),
        )
        
        # 3. Stream the final text response
        response_chunks = [
            "Based on the calculation, ",
            "7 multiplied by 6 ",
            f"equals {result}!"
        ]
        
        full_text = ""
        for chunk in response_chunks:
            full_text += chunk
            yield ResponsesAgentStreamEvent(
                **self.create_text_delta(delta=chunk, item_id="msg_1")
            )
        
        # 4. Yield complete text output
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item(
                text=full_text,
                id="msg_1",
            ),
        )


# Test streaming with tools
print("Testing streaming with tool calls...\n")
tool_agent = StreamingToolAgent()

print("Streaming events:")
print("=" * 60)

for event in tool_agent.predict_stream({"input": [{"role": "user", "content": "What is 7 times 6?"}]}):
    if event.type == "response.output_item.done":
        item = event.item
        if isinstance(item, dict):
            item_type = item.get("type")
        else:
            item_type = getattr(item, "type", "unknown")
        
        if item_type == "function_call":
            print(f"\n🔧 TOOL CALL: {item.get('name', getattr(item, 'name', 'unknown'))}")
            print(f"   Arguments: {item.get('arguments', getattr(item, 'arguments', ''))}")
        elif item_type == "function_call_output":
            print(f"\n📤 TOOL RESULT: {item.get('output', getattr(item, 'output', ''))}")
        elif item_type == "message":
            print(f"\n💬 FINAL MESSAGE (complete)")
    elif event.type == "response.output_text.delta":
        print(f"[delta] {event.delta}", end="", flush=True)

print("\n" + "=" * 60)
print("✅ Tool streaming complete!")

## 6. Reasoning Items in Streaming

ResponsesAgent supports streaming chain-of-thought reasoning.

In [ ]:
class ReasoningStreamAgent(ResponsesAgent):
    """
    Demonstrates streaming with reasoning/chain-of-thought.
    
    Shows:
    - Reasoning items for transparency
    - Multi-step thinking
    - Final answer after reasoning
    """
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs)
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        Stream reasoning followed by answer.
        """
        # Stream reasoning steps
        reasoning_steps = [
            "Let me analyze this step by step...\n",
            "1. First, I'll consider the key factors\n",
            "2. Then, I'll evaluate the options\n",
            "3. Finally, I'll provide my recommendation\n",
        ]
        
        full_reasoning = ""
        for step in reasoning_steps:
            full_reasoning += step
            yield ResponsesAgentStreamEvent(
                **self.create_text_delta(delta=step, item_id="reasoning_1")
            )
            time.sleep(0.2)  # Simulate thinking
        
        # Yield complete reasoning item
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_reasoning_item(
                text=full_reasoning,
                id="reasoning_1",
            ),
        )
        
        # Stream final answer
        answer_chunks = [
            "Based on my analysis, ",
            "I recommend proceeding with option A ",
            "because it offers the best balance ",
            "of cost and performance."
        ]
        
        full_answer = ""
        for chunk in answer_chunks:
            full_answer += chunk
            yield ResponsesAgentStreamEvent(
                **self.create_text_delta(delta=chunk, item_id="answer_1")
            )
        
        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item(
                text=full_answer,
                id="answer_1",
            ),
        )


# Test reasoning streaming
print("Testing reasoning stream agent...\n")
reasoning_agent = ReasoningStreamAgent()

print("Streaming with reasoning:")
print("=" * 60)

current_section = None
for event in reasoning_agent.predict_stream({"input": [{"role": "user", "content": "Should I use Python or JavaScript?"}]}):
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.output_item.done":
        item = event.item
        if isinstance(item, dict):
            item_type = item.get("type")
        else:
            item_type = getattr(item, "type", "unknown")
        
        if item_type == "reasoning":
            print("\n\n[💭 Reasoning Complete]\n")
            print("-" * 40)
            print("[📝 Now generating answer...]\n")

print("\n" + "=" * 60)
print("\n✅ Reasoning streaming complete!")

## 7. Complete Streaming Agent Example

A production-ready streaming agent combining all patterns.

In [ ]:
%%writefile streaming_agent.py
"""Production streaming agent with ResponsesAgent."""

import os
from typing import Generator

import mlflow
from mlflow.entities.span import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from openai import OpenAI


class StreamingChatAgent(ResponsesAgent):
    """
    Production-ready streaming chat agent.
    
    Features:
    - Real-time text streaming from OpenAI
    - Automatic format conversion
    - Full MLflow tracing
    - Custom output support
    """
    
    def __init__(self, model: str = "gpt-4o-mini", system_prompt: str = None):
        self.model = model
        self.client = OpenAI()
        self.system_prompt = system_prompt or "You are a helpful assistant."
    
    def stream_from_llm(self, messages: list) -> Generator:
        """Stream responses from OpenAI."""
        stream = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            stream=True,
        )
        for chunk in stream:
            yield chunk.to_dict()
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """Non-streaming prediction."""
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(
            output=outputs,
            custom_outputs=request.custom_inputs
        )
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """Stream response from LLM."""
        # Prepare messages with system prompt
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(
            to_chat_completions_input([i.model_dump() for i in request.input])
        )
        
        # Stream and convert
        yield from output_to_responses_items_stream(
            self.stream_from_llm(messages)
        )


# Enable tracing
mlflow.openai.autolog()

# Create and set model
agent = StreamingChatAgent(
    model="gpt-4o-mini",
    system_prompt="You are a helpful AI assistant. Be concise and friendly."
)
mlflow.models.set_model(agent)

In [ ]:
# Log the streaming agent
with mlflow.start_run(run_name="streaming_chat_agent") as run:
    model_info = mlflow.pyfunc.log_model(
        python_model="streaming_agent.py",
        artifact_path="agent",
        pip_requirements=[
            "mlflow",
            "openai",
            "pydantic>=2.0.0",
        ],
    )
    
    print(f"✅ Streaming agent logged!")
    print(f"Model URI: {model_info.model_uri}")

## 8. Production Considerations

### Streaming Best Practices:

1. **Always yield `output_item.done`**: Clients need complete items
2. **Consistent `item_id`**: All deltas for one item share the same ID
3. **Handle disconnections**: Clean up resources on client disconnect
4. **Timeout handling**: Set appropriate timeouts for long streams

### Error Handling:

In [ ]:
class RobustStreamingAgent(ResponsesAgent):
    """Agent with proper error handling for streaming."""
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict_stream(
        self, request: ResponsesAgentRequest
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        try:
            # Your streaming logic here
            yield ResponsesAgentStreamEvent(
                **self.create_text_delta(delta="Processing...", item_id="msg_1")
            )
            
            # Simulate work
            yield ResponsesAgentStreamEvent(
                type="response.output_item.done",
                item=self.create_text_output_item(
                    text="Processing complete!",
                    id="msg_1",
                ),
            )
            
        except Exception as e:
            # Yield error as a proper output item
            yield ResponsesAgentStreamEvent(
                type="response.output_item.done",
                item=self.create_text_output_item(
                    text=f"Error occurred: {str(e)}",
                    id="error_msg",
                ),
            )
    
    @mlflow.trace(span_type=SpanType.AGENT)
    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs)


print("✅ Robust streaming agent pattern demonstrated!")

## Summary

### What We Learned:

1. ✅ **Why streaming matters**: Better UX and real-time visibility
2. ✅ **Streaming interface**: Events, deltas, and output items
3. ✅ **Basic text streaming**: Yield deltas with consistent IDs
4. ✅ **OpenAI integration**: Using conversion utilities
5. ✅ **Tool call streaming**: Real-time tool execution visibility
6. ✅ **Reasoning streaming**: Chain-of-thought transparency
7. ✅ **Error handling**: Robust production patterns

### Key Helper Methods:

| Method | Purpose |
|--------|--------|
| `create_text_delta()` | Stream text chunks |
| `create_text_output_item()` | Complete text output |
| `create_function_call_item()` | Tool invocation |
| `create_function_call_output_item()` | Tool result |
| `create_reasoning_item()` | Chain-of-thought |
| `create_annotation_added()` | Annotations |

### Next Steps:
- Explore OpenAI Responses API integration
- Learn about deployment options
- Build production streaming applications